<a href="https://colab.research.google.com/github/IsaacFigNewton/DAG-Based-Compression/blob/main/DAG_Text_Encoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TODO


0.  Understand connected_components(adjacency_matrix, directed=True, return_labels=True) more in-depth
1.  Convert each token embedding in the intermediate process to a sparse matrix
2.  Refactor the suffix tree composition algorithm to take a divide-and-conquer aka a split-and-merge approach using multithreading
3.  Represent vectorization of text as fourth dimension, with token vector scaling corresponding to token frequencies or paths

# Install packages and import libraries

In [1]:
# !pip install tensorflow

In [2]:
import time
import re
from threading import Thread

import math
import random
from queue import Queue
import numpy as np
import urllib.request as url
import pandas as pd
import json

from scipy.sparse.csgraph import connected_components
import scipy.sparse as sp
import tensorflow as tf
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
# warnings.filterwarnings('ignore')

#Important functions

## Classes

### Storage

In [3]:
class DAGStore:
  def __init__(self,
               vertices=None,
               edge_set=None,
               token_index_map=None,
               adjacency_matrix=None):

      if vertices is None:
        vertices = dict()
      if edge_set is None:
        edge_set = set()

      self.vertices = vertices
      self.edge_set = edge_set
      self.token_index_map = token_index_map
      self.reversed_token_map = None
      # first dimension = outgoing token node
      # second dimension = incoming token node
      self.adjacency_matrix = adjacency_matrix

### SuffixNode Class
Let:
*    n=text length
*    b=mean block size
*    l=mean inter-block lexical diversity (l = n/b if every block has a unique suffix tree and l < n/b if some blocks have shared suffix tree compositions)

<br>

Blocked Ukkonen time complexity: O(n)
*   If split with preprocessing (O(n) time), parallelization in a divide-and-conquer approach would offer maximum time complexity of O(b*l)



pruning time complexity: O(log(b*l))

tokenization time complexity: O(log(b*l))

In [4]:
class SuffixNode:
    def __init__(self,
                 suffix=None,
                 token=None,
                 frequency=0,
                 parent=None,
                 children=None):
        self.suffix = suffix
        self.token = token
        self.frequency = frequency

        self.parent = parent

        if children is None:
            children = set()  # Initialize children as an empty set
        self.children = children

    def __str__(self):
        return self.suffix


    def split_edge(self, child, i):
      old_suffix = child.suffix[i:]
      new_suffix = self.suffix[i:]

      # Replace the old entry for the current node with a new one for the edge split
      split_node = SuffixNode(suffix=child.suffix[:i],
                              frequency=child.frequency,
                              parent=self,
                              children=child.children)

      # Remove the old child and add the new split node
      self.children.remove(child)
      self.children.add(split_node)

      # Create a new node for the existing edge suffix
      new_child = SuffixNode(suffix=old_suffix,
                              frequency=1,
                              parent=split_node,
                              children=split_node.children)

      # Transfer the children to the new split node's child
      for grandchild in split_node.children:
          grandchild.parent = new_child

      # Create a new node for the new suffix
      #     and move both it and the previous suffix beneath the split node
      split_node.children = {new_child,
                              SuffixNode(suffix=new_suffix,
                                        frequency=1,
                                        parent=split_node)
      }


    def add_suffix(self, suffix):

        # Find the longest prefix match in the children
        for child in self.children:

            # Find the index of the longest common prefix
            i = 0
            while i < len(child.suffix) and i < len(suffix) and child.suffix[i] == suffix[i]:
                i += 1

            # If there is a common prefix
            if i > 0:
                # Update the frequency of the current node
                child.frequency += 1

                # If the common prefix is the entire child key, recurse into that child
                if i == len(child.suffix):
                    child.add_suffix(suffix[i:])

                # If the common prefix is only part of the child key, split the edge
                elif i < len(suffix):
                    self.split_edge(child, i)

                return

        # No matching prefix, add the suffix as a new child
        self.children.add(SuffixNode(suffix=suffix,
                                     frequency=1,
                                     parent=self))


    def build_tree(self, text, do_blocking=True, delimeters=r"\n"):
        clauses = [text]
        # split the text into blocks, where each block is an independent clause
        if (do_blocking):
          clauses = re.split(delimeters, text)

        for string in clauses:
            # loop through the string, starting with the last character
            for i in range(0, len(string)):
                suffix = string[len(string) - i - 1:]

                # add the suffix to the tree
                self.add_suffix(suffix)


    def prune_tree(self, threshold=2):
        # If the node has no children (ie it's a leaf), return
        if not self.children:
            # print("no children")
            return

        children_to_kill = set()
        # Recursively prune the tree
        for child in self.children:

            if isinstance(child, SuffixNode):
                # if the child is above the threshold or it's a single character token node
                if child.frequency >= threshold or child.parent is None:
                    # print("not removed")
                    child.prune_tree(threshold)
                else:
                    # print("removed")
                    children_to_kill.add(child)

        for child in children_to_kill:
            # if the token's frequency falls below the threshold, prune it
            self.children.remove(child)


    # set all the suffix tree nodes' token properties
    # return an aggregated set of all the tokens
    def get_tokens(self, prev_token=""):
        tokens = dict()

        for child in self.children:
            if isinstance(child, SuffixNode):
                token = prev_token + child.suffix

                # set the child's token for later use
                child.token = token

                # add the accumulation of the current suffix with previous suffixes
                #   as a new token in the token set, with the previously found frequency
                tokens[token] = child.frequency
                # Recursively get tokens from the child
                tokens.update(child.get_tokens(token))

        return tokens

In [5]:
def build_suffix_tree(text,
                      threshold,
                      do_blocking=True,
                      delimeters=r"\n",
                      tree=None):
  # use the tree option in case a previous tree is to be pruned further
  suffix_tree = tree
  if suffix_tree is None:
    suffix_tree = SuffixNode(children = { SuffixNode(suffix=unique_char) for unique_char in set(text) })
    suffix_tree.build_tree(text,
                          do_blocking=do_blocking,
                          delimeters=delimeters)
  suffix_tree.prune_tree(threshold=threshold)

  tokens = suffix_tree.get_tokens()
  alphabet = list(set(text))

  return suffix_tree, tokens, { alphabet[i]: i for i in range(len(alphabet)) }

### CompositionDAGNode
construction time complexity: O(log(|V| + |E|))

In [6]:
class CompositionDAGNode:
    def __init__(self,
                 token=None,
                 frequency=0,
                 parents=None,
                 children=None,
                 dag_store=None):
        self.token = token
        self.frequency = frequency

        if parents is None:
            parents = list()
        self.parents = parents

        if children is None:
            children = set()
        self.children = children

        if dag_store is None:
            dag_store = DAGStore()
        self.dag_store = dag_store

    def __str__(self):
        child_tokens = {child.token for child in self.children}
        parent_tokens = {parent.token for parent in self.parents}
        return f"Token: {self.token}\nParents: {parent_tokens}\nChildren: {child_tokens}"

    # make an edge between the current token and a successor
    def add_edge(self, child):
        # Add the larger token as a child of this token
        self.children.add(child)

        # Add this token to the list of parents composing the larger token
        child.parents.append(self)

        # Add an edge to the edge list, using the current token's position
        #   in the child token as the edge weight
        self.dag_store.edge_set.add((self.token, child.token, len(child.parents) - 1))
        if self.token is not None and child.token is not None:
            self\
                .dag_store\
                .adjacency_matrix[self.dag_store.token_index_map[self.token], self.dag_store.token_index_map[child.token]] = 1


    # since this is recursively saving smaller tokens, it's basically depth-first
    def build_subgraph(self, all_tokens, suffix_tokenization):
      vertices = self.dag_store.vertices

      for token in suffix_tokenization:

        # if the predecessor token is not in the vertex store,
        #   recursively build a sub-graph of suffix tokens
        if token not in vertices.keys():
            # create a new dag node for the current token
            #   and put it in the vertex store
            vertices[token] = CompositionDAGNode(token=token,
                                                 frequency=all_tokens[token],
                                                 dag_store = self.dag_store)

            # break the missing token into even smaller tokens using the largest available smaller tokens
            curr_suffix_tokenization = tokenize(token, all_tokens, len(token) - 1)
            # build a subgraph from the smaller tokens
            temp_vert = vertices[token]
            temp_vert, additional_vertices = temp_vert.build_subgraph(all_tokens, curr_suffix_tokenization)
            vertices.update(additional_vertices)

            vertices[token] = temp_vert

        # base case: if the predecessor is in the vertex store
        #   add an edge from the current node's predecessor to it
        vertices[token].add_edge(self)

      return self, vertices


    # do breadth-first accumulation of the suffix tree into the dag
    def suffix_tree_to_dag(self, suffix_tree):
        all_tokens = suffix_tree.get_tokens()
        # create a dict for mapping tokens to indices in the adjacency matrix
        token_list = list(all_tokens.items())
        self.dag_store.token_index_map = {token_list[i][0]: i for i in range(len(token_list))}
        self.dag_store.reversed_token_map = {v: k for k, v in self.dag_store.token_index_map.items()}
        # initialize the adjacency matrix in LIL format
        #   for more efficient memory usage during composition and later modifications
        self.dag_store.adjacency_matrix = sp.lil_matrix((len(all_tokens), len(all_tokens)))

        vertices = self.dag_store.vertices
        vertices[self.token] = self

        suffix_node_queue = Queue()
        suffix_node_queue.put(suffix_tree)

        while not suffix_node_queue.empty():
            # get the next suffix node from the queue
            current_suffix_node = suffix_node_queue.get()

            # create a dag vertex and add it to the set of vertices
            vert = CompositionDAGNode(token=current_suffix_node.token,
                                      frequency=current_suffix_node.frequency,
                                      dag_store = self.dag_store)
            vertices[vert.token] = vert

            # if it's the root of the base dag or one of the top-level tokens, just add it to the vertex dict
            if current_suffix_node.parent is None or current_suffix_node.parent.token is None:
                vertices[current_suffix_node.token] = vert
                # add an edge from the base graph's root to the top-level token
                self.add_edge(vert)

            # otherwise, add edges
            else:
                # tokenize the current token using the largest available smaller tokens
                current_tokenization = tokenize(current_suffix_node.token,
                                               all_tokens,
                                               len(current_suffix_node.token) - 1)

                temp_vert = vert
                temp_vert, additional_vertices = temp_vert.build_subgraph(all_tokens, current_tokenization)
                vertices[vert.token] = temp_vert
                vertices.update(additional_vertices)

            # add all the current node's children to the queue
            for child in current_suffix_node.children:
                suffix_node_queue.put(child)

        # convert the LIL adjacency matrix to CSR format for more efficient modification
        self.dag_store.adjacency_matrix = sp.csr_matrix(self.dag_store.adjacency_matrix)

        self.dag_store.edge_set = {(pre, cum, pos) for pre, cum, pos in self.dag_store.edge_set if pre is not None}

##Utility

###Operational

In [7]:
def tokenize(text, token_dict, max_token_len):
    # filter to include only tokens shorter than the
    available_tokens = {token: freq for token, freq in token_dict.items() if len(token) <= max_token_len}

    # sort tokens by length and secondarily frequency
    best_tokens = sorted(list(available_tokens.items()),
                         key=lambda x: (-len(x[0]), -x[1]))

    # extract tokens from the ordered list
    tokenization = []
    while text != "":
        for token, freq in best_tokens:
            if text.startswith(token):
                tokenization.append(token)
                text = text[len(token):]

    return tokenization

### Vector Embedding

In [8]:
def calculate_distances_for_subgraph(labels, adjacency_matrix, subgraph_id):
    """
    Calculate Manhattan distances for nodes in a specific subgraph and update the distance matrix.

    Parameters:
    labels (numpy array): Array of subgraph labels for each node.
    adjacency_matrix (CSR matrix): A sparse adjacency matrix of the graph.
    subgraph_id (int): ID of the subgraph to process.
    """
    # extract a list of all the subgraph vertices in the current subgraph
    subgraph_vertices = np.where(labels == subgraph_id)[0]

    n = adjacency_matrix.shape[0]

    indices = []
    values = []
    # for each seed node
    for i in subgraph_vertices:
        # for each outgoing node in the subgraph
        for x in subgraph_vertices:
          # for each incoming node in the subgraph
          for y in subgraph_vertices:
              # if the entry, when double-checked, is found to be nonzero
              #   or, if the entry represents the seed node
              if adjacency_matrix[x, y] != 0 or i == x and x == y:
                  # calculate inverted Manhattan distance
                  inverse_manhattan_distance = 1 / np.maximum(np.abs(x - i) + np.abs(y - i), 1)

                  # Store indices and values in lists
                  indices.append([i, x, y])
                  values.append(inverse_manhattan_distance)

    if not indices:
        print("Error, the entry for the seed node was not created in the tensor")
        return None

    # return for recombination with the entire graph's tensor
    return indices, values

In [9]:
# def combine_sparse_tensors(tensors):
#     if tf.shape(tensors[0].values).numpy()[0] == 0:
#         # If tensor0 is empty, initialize it with tensor1
#         combined_sparse = tensors[1]
#     else:
#         indices = []
#         values = []
#         dense_shape = tensors[0].dense_shape

#         for tensor in tensors:
#             indices.append(tensor.indices)
#             values.append(tensor.values)

#         print(indices)
#         print(values)

#         # Flatten and concatenate indices and values
#         all_indices = tf.concat(indices, axis=0)
#         all_values = tf.concat(values, axis=0)

#         # Create a SparseTensor from the combined indices and values
#         combined_sparse = tf.sparse.SparseTensor(indices=all_indices, values=all_values, dense_shape=dense_shape)

#     #     # Use `tf.sparse.reduce_sum` to sum values at duplicate indices
#     #     combined_dense = tf.sparse.to_dense(combined_sparse)

#     # return tf.sparse.from_dense(combined_dense)
#     return combined_sparse

In [26]:
def vectorize_adjacency_matrix(adjacency_matrix, low_mem=True):
    """
    Vectorize the adjacency matrix by calculating Manhattan distances for each subgraph.

    Parameters:
    adjacency_matrix (CSR matrix): Adjacency matrix of the graph.

    Returns:
    sparse CSR matrix: 3D distance matrix.
    """
    n = adjacency_matrix.shape[0]

    # Identify subgraphs and their labels
    num_subgraphs, labels = connected_components(adjacency_matrix, directed=True, return_labels=True)
    indices = []
    values = []

    if low_mem:
        # Calculate distances for each subgraph in series
        for subgraph_id in range(num_subgraphs):
            subgraph_distance_tensor = calculate_distances_for_subgraph(labels,\
                                                                        adjacency_matrix,\
                                                                        subgraph_id)

            # if there's a subgraph distance tensor to combine the original with...
            if subgraph_distance_tensor is not None:
                # Recombine the partial vectorizations given by the subgraphs into
                #   a single vectorization for the entire graph
                # print(subgraph_distance_tensor)
                subgraph_indices, subgraph_values = calculate_distances_for_subgraph(labels,
                                                                                      adjacency_matrix,
                                                                                      subgraph_id)
                indices = indices + subgraph_indices
                values = values + subgraph_values
    else:
        # Calculate distances for each subgraph in parallel
        # Create and start threads for each subgraph
        threads = []
        # for subgraph_id in range(num_subgraphs):
        #     thread = Thread(target=calculate_distances_for_subgraph,
        #                     args=(labels,
        #                           adjacency_matrix,
        #                           subgraph_id))
        #     thread.start()
        #     threads.append(thread)

        # # Wait for all threads to complete
        # for thread in threads:
        #     thread.join()
        pass


    print(indices, values)

    # Return a sparse tensor for storing the tokens' distance tensors/embeddings
    return tf.sparse.SparseTensor(indices=indices,
                                  values=values,
                                  dense_shape=[n, n, n])

In [34]:
def tensor_to_csr(tensor):
  # Convert SparseTensor to COO format
  indices_np = np.array(tensor.indices.numpy())
  values_np = np.array(tensor.values.numpy())
  shape_np = tensor.dense_shape.numpy()

  print(shape_np)

  # Ensure indices_np contains pairs of row and column indices
  if len(indices_np.shape) == 1:
      # Reshape indices_np if it's one-dimensional
      indices_np = indices_np.reshape(-1, 2)

  coo = sp.coo_matrix((values_np, (indices_np[:, 0], indices_np[:, 1])), shape=(tensor.dense_shape[0], tensor.dense_shape[1]))

  # Convert COO matrix to CSR format and retrun the reformatted tensor
  return coo.tocsr()

In [35]:
def vectorize(adjacency_matrix, reversed_token_map, tokenizations):
  n = len(tokenizations)
  token_tensor = vectorize_adjacency_matrix(adjacency_matrix)
  # print(str(token_tensor)[:50])
  # print(composition_dag.dag_store.reversed_token_map)
  # create a dictionary of all the tokens and their respective tensor embedding slices
  # print(reversed_token_map)
  token_vector_mappings = {reversed_token_map[i]: tf.sparse.slice(token_tensor,
                                                                  start=[i, 0, 0],
                                                                  size=[1, token_tensor.shape[1], token_tensor.shape[2]]) for i in range(n)}

  print(token_vector_mappings)

  for key, token_set in tokenizations.items():
    for token in token_set.keys():
      print(token)
      tok_vect_tensor = token_vector_mappings[token]
      print(tok_vect_tensor)

      token_vector_mappings[token] = tensor_to_csr(token_vector_mappings[token])
      print(token_vector_mappings[token])

  return token_vector_mappings

### Plotting and Printing

In [13]:
def print_tree(tree, indent = 0):
    # Iterate over the child.suffixes (features) in the tree
    for child in tree.children:
        print(' ' * indent + str(child.suffix) + ": " + str(child.frequency))
        # If the child is a SuffixNode, recursively print the subtree
        if isinstance(child, SuffixNode):
            print_tree(child, indent + 4)
        else:
            print(' ' * (indent + 4) + "\"" + str(child.suffix) + "\": " + str(child.frequency))

In [14]:
def plot_embeddings(embeddings, max_plots):
  i = 0
  for token, vector in embeddings.items():
    if (i < max_plots):
        print("Embedding for'" + token + "':")

        # Create the heatmap
        sns.heatmap(vector, annot=False, cmap='viridis')

        # Display the heatmap
        plt.show()
        i += 1

In [15]:
def plot_dag(dag_store, A=None, edge_set=None, scaling=50, edge_width=1, k=2):

    if A is None:
        # Initialize a directed graph
        G = nx.DiGraph()
        # Add edges to the graph
        for parent, child, position in edge_set:
            G.add_edge(parent, child)#, weight=position)

    else:
        # Create a directed graph from the adjacency matrix
        G = nx.from_numpy_array(A, create_using=nx.DiGraph)

        # Relabel the token nodes
        G = nx.relabel_nodes(G, dag_store.reversed_token_map)

        # # Draw the graph without edge labels
        # # Convert nodes to strings before calculating length
        # nx.draw(G, with_labels=True, node_size=[scaling * len(str(node)) for node in G.nodes()],
        #         width=edge_width, font_size=8)


    # Calculate figure size based on the number of nodes
    num_nodes = len(G.nodes)
    num_edges = len(G.edges)
    graph_size = (num_nodes) + (2 * num_edges)

    figsize = graph_size * scaling/300
    font_size = 2 + math.sqrt(scaling)/5

    # Position nodes using the spring layout
    pos = nx.spring_layout(G, seed=42, k=k/num_nodes)

    # Calculate node sizes based on the length of the token text
    node_sizes = [scaling * len(node) for node in G.nodes()]

    plt.figure(figsize=(figsize, figsize), dpi=300)

    # Draw nodes with sizes proportional to the length of their text
    nx.draw_networkx_nodes(G, pos, node_size=node_sizes)

    # Draw edges with widths based on edge weights
    nx.draw_networkx_edges(G,
                          pos,
                          edgelist=edge_set,
                          width=edge_width,
                          arrowstyle='-|>',
                          connectionstyle="arc3,rad=0.2")

    # Draw node labels
    nx.draw_networkx_labels(G, pos, font_size=font_size, font_family="sans-serif")

    # # Draw edge weight labels
    # edge_labels = nx.get_edge_attributes(G, "weight")
    # nx.draw_networkx_edge_labels(G, pos, edge_labels)

    # Customize and show plot
    ax = plt.gca()
    ax.margins(0.08)
    plt.axis("off")
    plt.tight_layout()

    plt.show()

#Testing

## Variables and functions

###Testing function

In [16]:
def run_test(text,
             min_freq,
             blocking,
             delimeters=r"\n",
             tree=None):
  suffix_tree, tokenizations[(text, min_freq)], alphabet = build_suffix_tree(text,
                                                                 min_freq,
                                                                 do_blocking=blocking,
                                                                 delimeters=delimeters,
                                                                 tree=tree)
  # print("modified suffix tree composed in ", time.time() - start_time, " seconds.")
  # print_tree(suffix_tree)

  composition_dag = CompositionDAGNode()
  start_time = time.time()
  composition_dag.suffix_tree_to_dag(suffix_tree)
  end_time = time.time() - start_time
  # print("dag composed in ", end_time, " seconds.")

  if (min_freq % int(max(freq_range)/num_graphs_to_plot) == 0):
      plot_dag(composition_dag.dag_store,
               A=composition_dag.dag_store.adjacency_matrix,
               k=4,
               scaling=25)

  # get tensor embeddings for all vertices
  token_vector_mappings = vectorize(composition_dag.dag_store.adjacency_matrix,
                                    composition_dag.dag_store.reversed_token_map,
                                    tokenizations)

  # return {(pre, cum, pos+1) for pre, cum, pos in composition_dag.dag_store.edge_set if pre is not None}
  return end_time, suffix_tree, token_vector_mappings

###Variables

In [17]:
# freq_range = range(30, 1010, 10)
freq_range = range(5, 25, 5)
folds = 1
num_graphs_to_plot = 1
max_vector_plots = 2
tokenizations = dict()
test_results = {
    "min frequency": [],
    "test number": [],
    "blocking?": [],
    "mean time": [],
}

In [18]:
test_url = "https://courses.cs.washington.edu/courses/cse163/20wi/files/lectures/L04/bee-movie.txt"
with url.urlopen(test_url) as f:
    text = f.read().decode('utf-8')
# previously 454:500
# text = text[0:2000]
text = "abbabba"

tests = [text]

print(text[:50])

abbabba


## Run tests

In [36]:
# blocking or non-blocking
for j in range(2):
  print("blocking = ", bool(j))
  prev_trees = dict()

  for min_freq in freq_range:
    print("minimum frequency: ", min_freq)

    for i in range(len(tests)):
      # print("test ", i)
      # if there's no previous tree stored for this test
      if i not in prev_trees.keys():
        prev_trees[i] = None

      mean_times = [0, 0]
      for fold in range(folds):
        new_time=0
        new_time, prev_trees[i], token_vector_mappings = run_test(text=tests[i],
                                                          min_freq=min_freq,
                                                          blocking=bool(j),
                                                          delimeters=r"\n",     #r"\n\n|.\n|\)\n|:|\.\.\.",
                                                          tree=prev_trees[i])
        mean_times[j] += new_time

        plot_embeddings(token_vector_mappings, max_vector_plots)

      test_results["min frequency"].append(min_freq)
      test_results["test number"].append(i)
      test_results["blocking?"].append(j)
      test_results["mean time"].append(mean_times[j]/folds)

  print()

blocking =  False
minimum frequency:  5
[[0, 0, 0], [1, 1, 1]] [1.0, 1.0]
{'a': SparseTensor(indices=tf.Tensor([[0 0 0]], shape=(1, 3), dtype=int64), values=tf.Tensor([1.], shape=(1,), dtype=float64), dense_shape=tf.Tensor([1 2 2], shape=(3,), dtype=int64))}
a
SparseTensor(indices=tf.Tensor([[0 0 0]], shape=(1, 3), dtype=int64), values=tf.Tensor([1.], shape=(1,), dtype=float64), dense_shape=tf.Tensor([1 2 2], shape=(3,), dtype=int64))
[1 2 2]
  (0, 0)	1.0
b


KeyError: 'b'

In [ ]:
tests_df = pd.DataFrame.from_dict(test_results)
tests_df.head()

In [ ]:
noblock = tests_df[tests_df["blocking?"] == 0]
block = tests_df[tests_df["blocking?"] == 1]
plt.plot(noblock["min frequency"],
          noblock["mean time"],
          "r",
          block["min frequency"],
          block["mean time"],
          "g"
)

plt.xscale('log')
plt.yscale('log')

plt.xlabel('Min Frequency (log scale)')
plt.ylabel('Mean Time (log scale)')
plt.show()